In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train =train.replace(0,0.4)

In [ ]:
col = [c for c in train.columns if c not in ['ID', 'target']]
print(train.shape, test.shape)
print(train.columns)
print(col)


In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()

In [ ]:
def rmsle(y, pred):
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(pred), 2)))


In [ ]:
from sklearn.model_selection import train_test_split
x1, x2, y1, y2 = train_test_split(train[col], train.target.values, test_size=0.20, random_state=5)

In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
x11 = scl.fit_transform(x1)
x22 = scl.transform(x2)

In [ ]:
print(x22.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs = -1, random_state = 7)
model.fit(x11, y1)
pr = model.predict(x22)
print(rmsle(y2,pr))
            

In [ ]:
test['target_lgb'] = 0.0
test['target_xgb'] = 0.0
folds = 10

In [ ]:
from sklearn.model_selection import train_test_split
for fold in range(folds):
    x1, x2, y1, y2 = train_test_split(train[col], np.log1p(train.target.values), test_size=0.20, random_state=fold)

In [ ]:
print(x1.head)

In [ ]:
print(x2.head)

In [ ]:
print(y1)

In [ ]:
print(y2)

In [ ]:
#LightGBM
import lightgbm as lgb
params = {'learning_rate': 0.02,'max_depth': 13,'reg_alpha':0.04,'reg_lambda':0.073,'boosting': 'gbdt','objective': 'regression','metric': 'rmse','is_training_metric': True, 'num_leaves': 12**2,'feature_fraction': 0.9,'bagging_fraction': 0.8, 'bagging_freq': 5,'min_split_gain':0.0222415,'min_child_weight':40,'subsample':0.8715623,'seed':fold,'silent':-1,'verbose':-1}
model = lgb.train(params, lgb.Dataset(x1, label=y1), 3000, lgb.Dataset(x2, label=y2), verbose_eval=200, early_stopping_rounds=100)
test['target_lgb'] += np.expm1(model.predict(test[col], num_iteration=model.best_iteration))

In [ ]:
 #XGB
#referred: www.kaggle.com/samratp/santander-value-prediction-xgb-and-lightgbm 
import xgboost as xgb
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'eta': 0.005, 'max_depth': 10, 'subsample': 0.7, 'colsample_bytree': 0.5, 'alpha':0, 'silent': True, 'random_state':fold}
model = xgb.train(params, xgb.DMatrix(x1, y1), 5000,  watchlist, maximize=False, verbose_eval=200, early_stopping_rounds=100)
test['target_xgb'] += np.expm1(model.predict(xgb.DMatrix(test[col]), ntree_limit=model.best_ntree_limit))

In [ ]:
test['target_lgb'] /= folds
test['target_xgb'] /= folds
test['target'] = (test['target_lgb'] + test['target_xgb'])/2
test[['ID', 'target']].to_csv('try.csv', index=False)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission["target"] = test['target']

In [ ]:
submission.shape

In [ ]:
print(submission.head)